In [2]:
# %pip install spotipy

In [1]:
from api_info import *
import torch
import torch.nn as nn
import numpy as np

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
# cid = '733feec7461347549s6335bd86b89e056'
# print('enter the secret value')
# secret = input()

In [3]:
# potential things to analyze:
# thom yorke vs carti lyrics (genius api)
# valence (happiness) vs popularity (data science)
# track features -> genre
# is [x] shoegaze/metal/emo/etc?

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

# only_shallow_id = '52UcjsM15hjCQAUbTW2hy1'
# songs = [only_shallow_id]

In [88]:


non_shoegaze_artists = [ #no shoegaze in entire discog!
    'Coldplay',
    'Kendrick Lamar',
    'Taylor Swift',
    'BTS',
    'Fleet Foxes',
    'Kanye West',
    'Black Country, New Road',
    'The Strokes',
    'Joy Division'
]

shoegaze_artists = [
    'my bloody valentine',
    'Slowdive',
    'Drop Nineteens',
    'Parannoul',
    'Swervedriver',
    'Ride',
    'LSD and the Search for God',
    'Candy Claws'
]

shoegaze_albums = [
    ('Loveless', 'my bloody valentine'),
    ('m b v', 'my bloody valentine'),
    ('Isnt Anything', 'my bloody valentine'),
    ('To See the Next Part of the Dream', 'Parannoul'),
    ('Souvlaki', 'Slowdive'),
    ('Just For a Day', 'Slowdive'),
    ('Delaware', 'Drop Nineteens'),
    ('Nowhere', 'Ride'),
    ('LSD and the Search for God', 'LSD and the Search for God'),
    ('Raise', 'Swervedriver'),
    ('Ceres & Calypso in the Deep Time', 'Candy Claws')
]

artists = non_shoegaze_artists + shoegaze_artists

In [89]:
songs = []

for artist in non_shoegaze_artists:
    results = sp.search(q="artist:" + artist, type="artist")
    artist_id = results['artists']['items'][0]['id']
    # print(artist['name'], artist['genres'])
    albums = sp.artist_albums(artist_id)['items']

    for album in albums:
        album_id = album['id']
        tracks = sp.album_tracks(album_id)['items']
        songs.extend([track['id'] for track in tracks])

In [90]:
for album, artist in shoegaze_albums:
    results = sp.search(q='artist:' + artist + ' album:' + album, type='album')
    album_id = results['albums']['items'][0]['id']
    tracks = sp.album_tracks(album_id)['items']
    songs.extend([track['id'] for track in tracks])

In [91]:
print(songs[10:15])
print(songs[-5:-1])

['2wleRXcIPqltoDPLMH5WDa', '0jH7gF7KCk2Lom9gimaKms', '7H4NC6MYkcO91vrXwRQ4N1', '4nb8OcZG8lpnHi5DmkEnY2', '49YD8pGETf5HUKxouVdq3W']
['3GSfVEFVJCTvtxF0UUzdIr', '0MjmarACqIH451S26Wfpuw', '74ctB0ufdAY69TDN9J18V9', '2w2WdQb59ERxKh25nFjyAx']


In [92]:
# for each song, make a list of:
# danceability, energy, speechiness, acousticness, instrumentalness, liveness, valence
# 1 if shoegaze, 0 if not

def encode_song(song_id):
    f = sp.audio_features(song_id)[0]
    return np.array([f['danceability'], f['energy'], f['speechiness'], f['acousticness'], f['instrumentalness'], f['liveness'], f['valence']])

def is_shoegaze(song_id):
    # artist_id = sp.track(song_id)['artists'][0]['id']
    # print(artist_id)
    # return int('shoegaze' in sp.artist(artist_id)['genres'])
    artist_name = sp.track(song_id)['artists'][0]['name']
    return int(artist_name in shoegaze_artists)

In [93]:
song_data = [encode_song(song) for song in songs]
genres = [is_shoegaze(song) for song in songs]

In [94]:
print(song_data[40:45])
print(genres[40:45])

print(song_data[-5:-1])
print(genres[-5:-1])

print(len(song_data))
print(len(genres))

[array([0.491 , 0.262 , 0.0572, 0.89  , 0.288 , 0.207 , 0.338 ]), array([0.476 , 0.18  , 0.0346, 0.943 , 0.235 , 0.138 , 0.236 ]), array([0.393 , 0.301 , 0.0497, 0.758 , 0.764 , 0.161 , 0.0384]), array([0.503  , 0.594  , 0.0263 , 0.00276, 0.0102 , 0.109  , 0.168  ]), array([0.512 , 0.452 , 0.0292, 0.775 , 0.0818, 0.065 , 0.235 ])]
[0, 0, 0, 0, 0]
[array([0.245 , 0.52  , 0.0398, 0.0535, 0.412 , 0.272 , 0.695 ]), array([0.145 , 0.628 , 0.0347, 0.186 , 0.835 , 0.128 , 0.222 ]), array([0.296 , 0.55  , 0.0427, 0.0024, 0.0654, 0.327 , 0.305 ]), array([0.281  , 0.694  , 0.0412 , 0.586  , 0.00712, 0.0868 , 0.334  ])]
[1, 1, 1, 1]
2524
2524


In [95]:
model = nn.Sequential(
  nn.Linear(7, 128),
  nn.ReLU(),
  nn.Linear(128, 128),
  nn.ReLU(),
  nn.Linear(128, 1),
  nn.Sigmoid()
)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)




In [96]:
data = []
for i in range(len(song_data)):
  data.append((song_data[i], genres[i]))

In [97]:
# print(len(x))
print(data[60:62])

[(array([0.512 , 0.452 , 0.0292, 0.775 , 0.0818, 0.065 , 0.235 ]), 0), (array([5.50e-01, 7.96e-01, 3.78e-02, 1.07e-03, 1.18e-05, 1.23e-01,
       5.10e-01]), 0)]


In [98]:
# import tqdm
for epoch in range(1, 51):
  losses = []
  for x, y in data:
    x = torch.tensor(x).reshape(7).float()
    y = torch.tensor(y).reshape(1).float()

    guess = model(x)

    loss = nn.functional.binary_cross_entropy(guess, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


    losses.append(loss.item())
  if epoch % 10 == 0:
    print("LOSS", np.mean(losses))

LOSS 0.11353819126685935
LOSS 0.09791673203887272
LOSS 0.091681720549452
LOSS 0.08702002040615807
LOSS 0.08325949012808677


In [99]:
def get_genre(name, artist):
  results = sp.search(q="track:" + name + " artist:" + artist, type="track")
  track_id = results['tracks']['items'][0]['id']

  song = encode_song(track_id)
  song = torch.tensor(song).float().reshape(7)

  value = model(song)[0].item()
  probability = round(10000 * value) / 100
  if round(value):
    return f'{name} by {artist} is shoegaze ({probability}% confidence)'
  return f'{name} by {artist} is not shoegaze ({100 - probability}% confidence)'


In [101]:
test_songs = [
    ("New Slaves", 'Kanye West'),
    ("Self Control", 'Frank Ocean'),
    ("Let Down", 'Radiohead'),
    ("Blow Out", 'Radiohead'),
    ('Only Shallow', 'my bloody valentine'),
    ('Heaven or Las Vegas', 'Cocteau Twins'),
    ('Fell in Love (At the Water)', 'Candy Claws'),
    ('Chocolate Matter', 'Sweet Trip'),
    ('Bloodhail', 'Have a Nice Life'),
    ('Vapour Trail', 'Ride'),
    ('Hell of a Life', 'Kanye West'),
    ('Blank Space', 'Taylor Swift'),
    ('Alison', 'Slowdive'),
    ('Sometimes', 'my bloody valentine'),
    ('Beautiful World', 'Parannoul'),
    ('Modern Love Stories', 'Beach House'),
    ('Wesley\'s Theory', 'Kendrick Lamar'),
    ('Pictures of You', 'The Cure'),
    ('Blood Section', 'Swans'),
    ('Ring a Bell', 'Death Grips'),
    ('Total Trash', 'Sonic Youth'),
    ('The Scientist', 'Coldplay'),
    ('Territorial Pissings', 'Nirvana'),
    ('Hard to Explain', 'The Strokes'),
    ('Snow Globes', 'Black Country, New Road')
]

tests = [get_genre(x[0], x[1]) for x in test_songs]
for test in tests:
  print(test)

New Slaves by Kanye West is not shoegaze (95.97% confidence)
Self Control by Frank Ocean is not shoegaze (99.77% confidence)
Let Down by Radiohead is not shoegaze (76.86% confidence)
Blow Out by Radiohead is shoegaze (58.14% confidence)
Only Shallow by my bloody valentine is shoegaze (87.97% confidence)
Heaven or Las Vegas by Cocteau Twins is not shoegaze (65.85% confidence)
Fell in Love (At the Water) by Candy Claws is not shoegaze (62.65% confidence)
Chocolate Matter by Sweet Trip is not shoegaze (89.97% confidence)
Bloodhail by Have a Nice Life is not shoegaze (72.56% confidence)
Vapour Trail by Ride is shoegaze (79.83% confidence)
Hell of a Life by Kanye West is not shoegaze (91.37% confidence)
Blank Space by Taylor Swift is not shoegaze (98.98% confidence)
Alison by Slowdive is shoegaze (84.4% confidence)
Sometimes by my bloody valentine is shoegaze (85.59% confidence)
Beautiful World by Parannoul is shoegaze (87.97% confidence)
Modern Love Stories by Beach House is shoegaze (86.6